In [56]:
import rdflib
from rdflib import FOAF, RDF, URIRef, Literal, SKOS, Namespace

# Creating example patient demographics data

In [107]:
# I'm making up a new namespace to define the property 'id'

CUSTOM = Namespace('http://example.org/customNamespace')

Alice = URIRef('http://example.org/people/Alice')
Heidi = URIRef('http://example.org/people/Heidi')

g = rdflib.Graph()

g.add((Alice, CUSTOM.id, Literal('12345')))
g.add((Alice, RDF.type, FOAF.Person))
g.add((Alice, FOAF.name, Literal('Alice Williams')))

g.add((Heidi, CUSTOM.id, Literal('5678')))
g.add((Heidi, RDF.type, FOAF.Person))
g.add((Heidi, FOAF.name, Literal('Heidi Cook')))

# Querying the patients
For this quick prototype we're assuming that the URI of the elements serve as patient ids

In [117]:
query = '''
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX custom: <http://example.org/customNamespace>

SELECT ?id ?name 
WHERE{
?person a foaf:Person ;
    foaf:name ?name ;
    custom:id ?id
}
'''

In [119]:
result = g.query(query)

for r in g.query(query):
    for k, v in r.asdict().items():
        print(f'{k}: {v}')
        
    print('')

id: 12345
name: Alice Williams

id: 5678
name: Heidi Cook



In [120]:
from fhir.resources.patient import Patient
from fhir.resources.address import Address
from fhir.resources.fhirtypes import Date
from fhir.resources.humanname import HumanName

In [121]:
def create_patient(properties):
    # Some items in the patient resource require some extra work
    properties = dict(properties)
    properties['name'] = [HumanName(given=[properties['name']])]
    
    
    return Patient(**properties)

In [124]:
def create_patients():
    for r in result:
        d = r.asdict()
        patient = create_patient(d)
        yield patient
        
        
patients = list(create_patients())
for p in patients:
    print(p.json())

{"id": "12345", "name": [{"given": ["Alice Williams"]}], "resourceType": "Patient"}
{"id": "5678", "name": [{"given": ["Heidi Cook"]}], "resourceType": "Patient"}
